In [1]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
!pip install tensorflow-model-optimization
import tensorflow_model_optimization as tfmot
import zlib

"""
parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, required=True, help='model name')
parser.add_argument('--labels', type=int, required=True, help='model output')
args = parser.parse_args()"""


seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True,
    cache_dir='.', cache_subdir='data')
csv_path, _ = os.path.splitext(zip_path)
df = pd.read_csv(csv_path)

column_indices = [2, 5]
columns = df.columns[column_indices]
data = df[columns].values.astype(np.float32)

n = len(data)
train_data = data[0:int(n*0.7)]
val_data = data[int(n*0.7):int(n*0.9)]
test_data = data[int(n*0.9):]

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)

input_width = 12 


class WindowGenerator:
    def __init__(self, input_width, mean, std):
        self.input_width = input_width 
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        
        inputs = features[:, :6, :]
        labels = features[:, -6:, :]
        inputs.set_shape([None, 6, 2])
        labels.set_shape([None, 6, 2])

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)

        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
                data=data,
                targets=None,
                sequence_length=12,
                sequence_stride=1,
                batch_size=32)
        
        
        ds = ds.map(self.preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        ds = ds.cache()
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds.prefetch(tf.data.experimental.AUTOTUNE)


generator = WindowGenerator(input_width, mean, std)
train_ds = generator.make_dataset(train_data, True)
val_ds = generator.make_dataset(val_data, False)
test_ds = generator.make_dataset(test_data, False)

for x, y in train_ds:
    input_shape = x.shape.as_list()[1:]
    output_shape = y.shape.as_list()[1:]
    break

print(f'Input shape: {input_shape}')
print(f'Output shape: {output_shape}')

Input shape: [6, 2]
Output shape: [6, 2]


In [2]:
class MyModel:
    def __init__(self, model_name, alpha, input_shape, output_shape, final_sparsity=None):
        
        if model_name.lower() == 'mlp':
            # create the mlp model
            model = tf.keras.Sequential([
                tf.keras.layers.Flatten(input_shape=input_shape, name='flatten'),
                tf.keras.layers.Dense(128, activation='relu', name='first_dense'),
                tf.keras.layers.Dense(128, activation='relu', name='second_dense'),
                tf.keras.layers.Dense(12, activation='relu', name='third_dense'),
                tf.keras.layers.Reshape(output_shape)

            ])

        elif model_name.lower() == 'cnn':
            # create the cnn model
            model = tf.keras.Sequential([
                tf.keras.layers.Conv1D(input_shape = input_shape, filters=64, kernel_size=3, activation='relu', name='conv1d'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(64, activation='relu', name='first_dense'),
                tf.keras.layers.Dense(12, name='second_dense'),
                tf.keras.layers.Reshape(output_shape)
            ])
        
        model.summary()
        self.model = model
        self.alpha = alpha
        self.final_sparsity = final_sparsity
        self.model_name = model_name.lower()
        if alpha != 1:
            self.model_name += '_ws' + str(alpha).split('.')[1]
        if final_sparsity is not None and 'lstm' not in self.model_name :
            self.model_name += '_mb' + str(final_sparsity).split('.')[1]
            self.magnitude_pruning = True
        else:
            self.magnitude_pruning = False
        
        

    def compile_model(self, optimizer, loss_function, eval_metric):

        if self.magnitude_pruning:
            #sparsity scheduler
            pruning_params = {
                'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay( 
                                                                initial_sparsity=0.30,
                                                                final_sparsity=self.final_sparsity,
                                                                begin_step=len(train_ds)*5,
                                                                end_step=len(train_ds)*15)
            }

            prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
            self.model = prune_low_magnitude(self.model, **pruning_params)

            input_shape = [32, 6, 2]
            self.model.build(input_shape)

        self.model.compile(
            optimizer = optimizer,
            loss = loss_function,
            metrics = eval_metric
        )


        
    def train_model(self,X_train, X_val, N_EPOCH, callbacks=[]):
        
        if self.magnitude_pruning:
            callbacks.append(tfmot.sparsity.keras.UpdatePruningStep())

        print('\tTraining... ')
        print('\t', end='')

        history = self.model.fit(
            X_train, 
            epochs=N_EPOCH, 
            validation_data =X_val, 
            verbose=1,
            callbacks=callbacks,
        )
            
        return history
    
    def evaluate_model(self, X_test):
        return self.model.evaluate(X_test)
        
        
    def get_model(self):
        return self.model
    
    def save_model(self, model_folder):
        
        run_model = tf.function(lambda x: self.model(x))
        concrete_func = run_model.get_concrete_function(tf.TensorSpec([1, 6, 2], tf.float32))
        self.model.save(model_folder, signatures=concrete_func)
        print(f'Model {self.model_name} saved at {model_folder}')
    
    def convert_to(self, model_folder, tflite=True, weights_only=True, weights_activation=True):
        
        tflite_model_dir = os.path.join("./tflite_models", self.model_name)
        if not os.path.isdir(tflite_model_dir):
            os.makedirs(tflite_model_dir)
        
        print(f'From {model_folder} to {tflite_model_dir}')
        size_original_model = compute_size(model_folder)
        print(f'Size of the original {self.model_name}: {size_original_model} KB')
        
        if self.magnitude_pruning:

            self.model = tfmot.sparsity.keras.strip_pruning(self.model)
            converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
            
            tflite_model = converter.convert()
            with open(tflite_model_dir +'/saved_model.tflite', 'wb') as fp:
                tflite_compressed = zlib.compress(tflite_model) 
                fp.write(tflite_model)

        if tflite and not self.magnitude_pruning:  
            # --------- with tflite conversion
            converter = tf.lite.TFLiteConverter.from_saved_model(model_folder)
            # convert the model into a tflite version
            tflite_model = converter.convert()
            # stored in tflite_model_dir


            with open(tflite_model_dir+'/saved_model.tflite', 'wb') as fp: 
                fp.write(tflite_model)

            size_tflite_model = compute_size(tflite_model_dir)
            print(f'Size of the tflite {self.model_name}: {size_tflite_model} KB')

        if weights_only:
            
            qtflite_model_dir = os.path.join("./weight_only_PTQ_models", self.model_name)
            #if not os.path.isdir(qtflite_model_dir):
                #os.makedirs(qtflite_model_dir)
            
            # --------- with tflite quantization weight only
            converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            # convert the model into a tflite version
            tflite_model = converter.convert()

            with open(qtflite_model_dir + '/saved_model.tflite', 'wb') as fp:
                fp.write(tflite_model)

            size_qtflite_model = compute_size(qtflite_model_dir)
            print(f'Size of the weight only quantization model {self.model_name}: {size_qtflite_model} KB')

        
        if weights_activation and not (self.model_name=='lstm' or self.model_name=='cnn') :
            
            qatflite_model_dir = os.path.join("./weight_activation_PTQ_models", self.model_name)
            if not os.path.isdir(qatflite_model_dir):
                os.makedirs(qatflite_model_dir)

            # ---------  with tflite quantization weight and activation
            converter = tf.lite.TFLiteConverter.from_saved_model(model_folder)
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.representative_dataset = representative_dataset_gen

            # to force it to use only int8 ops, as well as int8 inputs and outputs
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
            converter.inference_input_type = tf.uint8
            converter.inference_output_type = tf.uint8
            # convert the model into a tflite version
            tflite_model = converter.convert()

            with open(qatflite_model_dir + '/saved_model.tflite', 'wb') as fp:
                fp.write(tflite_model)

            size_aqtflite_model = compute_size(qatflite_model_dir)
            print(f'Size of the weight and activation quantization model  {self.model_name}: {size_aqtflite_model} KB')

In [3]:
class CustomMAE(tf.keras.metrics.Metric):
    def __init__(self, name='CustomMAE', **kwargs):
        super(CustomMAE, self).__init__(name=name, **kwargs)
        self.sum = self.add_weight(shape=[2], name='sum', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # accumulate at each batch
        values = tf.cast(tf.abs(y_true - y_pred), dtype=tf.float32)
        print(values.shape)
        self.sum.assign_add(tf.reduce_mean(values, axis=[0, 1]))
        self.count.assign_add(1)

    def result(self):
        return tf.math.divide_no_nan(self.sum, self.count)

    def reset_states(self):
        self.sum.assign(tf.zeros_like(self.sum))
        self.count.assign(tf.zeros_like(self.count))


def compute_size(path_to_explore):
    size = 0
    for path in list(os.walk(path_to_explore)):
        
        root = path[0]
        files = path[2]
        for file in files:
            size += os.path.getsize(root + "/" + file)
        
    return round(size/1024, 3)

In [ ]:
N_EPOCH = 20

eval_metric = [CustomMAE()]
loss_function = [tf.keras.losses.MeanSquaredError()]
optimizer = tf.keras.optimizers.Adam()

for model_name in ['cnn']:
    alpha = 1
    model = MyModel(model_name, alpha, input_shape, output_shape, final_sparsity=0.9)
    model.compile_model(optimizer, loss_function, eval_metric)
    history = model.train_model(train_ds, val_ds, N_EPOCH, callbacks=[])

    [test_loss, test_mae] = model.evaluate_model(test_ds)
    print(test_mae)
    model_dir = f'./models/{model.model_name}'
    model.save_model(model_dir)
    model.convert_to(model_dir, tflite=True, weights_only=True, weights_activation=False)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 4, 64)             448       
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
first_dense (Dense)          (None, 64)                16448     
_________________________________________________________________
second_dense (Dense)         (None, 12)                780       
_________________________________________________________________
reshape (Reshape)            (None, 6, 2)              0         
Total params: 17,676
Trainable params: 17,676
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Please use `layer.add_weight` method instead.
	Training... 
	Epoch 1/20
(None, 6, 2)
(None,